In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# === Config ===
ticker = "AAPL"
surprise_threshold = 5  # in percent
holding_days = 5

# === Fetch earnings calendar from yfinance
stock = yf.Ticker(ticker)
earnings = stock.earnings
earnings_dates = stock.earnings_dates

# Check and use available earnings surprise data
try:
    cal = stock.calendar
    print("Calendar Info:\n", cal)
except:
    print("No calendar data available.")

# === Download price data
price_data = yf.download(ticker, start="2018-01-01", end="2025-01-01")['Close']

# === Load earnings history from Ticker.earnings
earnings = stock.earnings.reset_index()
earnings.columns = ['Year', 'Revenue', 'Earnings']

# Manual override if earnings surprises not available via yfinance
# Example placeholder earnings surprise dataframe:
earnings_surprise = pd.DataFrame({
    'date': pd.to_datetime([
        '2023-01-26', '2023-04-27', '2023-07-27', '2023-11-02',
        '2024-02-01'
    ]),
    'surprise_pct': [10.1, -7.3, 6.2, -5.4, 12.5]  # Example data
})

# === Signal Generation
signals = []
for _, row in earnings_surprise.iterrows():
    date = row['date']
    surprise = row['surprise_pct']
    signal = 1 if surprise > surprise_threshold else -1 if surprise < -surprise_threshold else 0

    if signal == 0 or date not in price_data.index:
        continue

    try:
        entry_price = price_data.loc[date]
        exit_date = price_data.index[price_data.index.get_loc(date) + holding_days]
        exit_price = price_data.loc[exit_date]
        ret = (exit_price - entry_price) / entry_price * signal
        signals.append({
            'entry_date': date,
            'exit_date': exit_date,
            'return': ret,
            'signal': signal,
            'surprise': surprise
        })
    except:
        continue

df_returns = pd.DataFrame(signals)
df_returns['cumulative_return'] = (1 + df_returns['return']).cumprod()

# === Plot Results
plt.figure(figsize=(10, 5))
plt.plot(df_returns['entry_date'], df_returns['cumulative_return'], marker='o')
plt.title(f'Alpha 12: Earnings Surprise Strategy — {ticker}')
plt.xlabel('Earnings Date')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.tight_layout()
plt.show()

# === Summary
print("Total Trades:", len(df_returns))
print("Average Return per Trade: {:.2f}%".format(df_returns['return'].mean() * 100))
print("Cumulative Return: {:.2f}x".format(df_returns['cumulative_return'].iloc[-1]))


/home/codespace/.python/current/lib/python3.12/site-packages/yfinance/scrapers/fundamentals.py:33: DeprecationWarning: 'Ticker.earnings' is deprecated as not available via API. Look for "Net Income" in Ticker.income_stmt.
  warnings.warn("'Ticker.earnings' is deprecated as not available via API. Look for \"Net Income\" in Ticker.income_stmt.", DeprecationWarning)
[*********************100%***********************]  1 of 1 completed

Calendar Info:
 {'Dividend Date': datetime.date(2025, 2, 13), 'Ex-Dividend Date': datetime.date(2025, 2, 10), 'Earnings Date': [datetime.date(2025, 5, 1)], 'Earnings High': 1.67, 'Earnings Low': 1.47, 'Earnings Average': 1.61038, 'Revenue High': 95903397000, 'Revenue Low': 89400000000, 'Revenue Average': 93961131880}



/home/codespace/.python/current/lib/python3.12/site-packages/yfinance/scrapers/fundamentals.py:33: DeprecationWarning: 'Ticker.earnings' is deprecated as not available via API. Look for "Net Income" in Ticker.income_stmt.
  warnings.warn("'Ticker.earnings' is deprecated as not available via API. Look for \"Net Income\" in Ticker.income_stmt.", DeprecationWarning)


AttributeError: 'NoneType' object has no attribute 'reset_index'